In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [22]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt


from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import create_dir, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter, read_saved_epochs
from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from tf_lab.point_clouds.raw_gan import RawGAN
from tf_lab.point_clouds.latent_gan import LatentGAN
from tf_lab.point_clouds.generators_discriminators import latent_code_discriminator_two_layers, \
latent_code_generator_two_layers, latent_code_discriminator, latent_code_generator

from tf_lab.iclr.helper import load_multiple_version_of_pcs, find_best_validation_epoch_from_train_stats
from tf_lab.point_clouds.convenience import get_latent_codes
from tf_lab.neural_net import MODEL_SAVER_ID

from tf_lab.point_clouds.raw_gan_gp import RawGAN_GP
from tf_lab.point_clouds.generators_discriminators import latent_code_generator

from pcloud_benchmark.evaluate_gan import entropy_of_occupancy_grid, jensen_shannon_divergence
from tf_lab.nips.helper import pclouds_centered_and_half_sphere

PyTorch not working. MMD measurement won't be available


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:
n_pc_points = 2048
do_training = True

save_model = False
saver_step = None

save_model = False
save_synthetic_samples = np.hstack([np.array([1,5,10]), np.arange(100, 2001, 100)])

ae_loss = 'emd'
do_lwgan = True

class_name = raw_input('Give me the class type.\n').lower()
syn_id = snc_category_to_synth_id()[class_name]
b_neck = int(raw_input('Give me bneck_size.\n'))

In [12]:
if do_lwgan:
    gan_tag = 'l_wgan'
else:
    gan_tag = 'l_gan'

In [14]:
special_tag = 'disc_512_1024'

In [15]:
experiment_tag = '_'.join([gan_tag, class_name, special_tag, ae_loss, 'bneck', str(b_neck)])
print experiment_tag

l_wgan_chair_disc_512_1024_emd_bneck_128


In [17]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models', experiment_tag)
create_dir(train_dir)

'/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/l_wgan_chair_disc_512_1024_emd_bneck_128'

In [18]:
in_data = load_multiple_version_of_pcs('uniform_one', syn_id, n_classes=1)

Loading test data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt
679 pclouds were loaded. They belong in 1 shape-classes.
Loading train data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt
5761 pclouds were loaded. They belong in 1 shape-classes.
Loading val data.
/orions4-zfs/projects/optas/DATA/Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt
338 pclouds were loaded. They belong in 1 shape-classes.


In [19]:
# Load Auto-Encoder
ae_experiment_tag = 'mlp_with_split_3pc_usampled_bnorm_on_encoder_only'
ae_id = '_'.join(['ae', class_name, ae_experiment_tag, str(n_pc_points), 'pts', str(b_neck), 'bneck', ae_loss])
ae_train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/', ae_id)

ae_conf = Conf.load(osp.join(ae_train_dir, 'configuration'))
saved_epochs = read_saved_epochs(ae_conf.train_dir)

_, best_epoch = find_best_validation_epoch_from_train_stats(osp.join(ae_train_dir, 'train_stats.txt'))

if best_epoch % ae_conf.saver_step != 0: # Model was not saved at that epoch.
    best_epoch += best_epoch % ae_conf.saver_step

ae_conf.encoder_args['verbose'] = False
ae_conf.decoder_args['verbose'] = False

reset_tf_graph()
ae = PointNetAutoEncoder(ae_conf.experiment_name, ae_conf)    
ae.restore_model(ae_conf.train_dir, best_epoch, verbose=True)

Model restored in epoch 430.


In [20]:
# Convert raw-data to latent codes.
latent_codes = get_latent_codes(ae, in_data['train'].point_clouds)
train_data = PointCloudDataSet(latent_codes)

In [21]:
init_lr = 0.0001
batch_size = 50
noise_params = {'mu':0, 'sigma': 0.2}
noise_dim = b_neck
max_epochs = 2000

n_syn_samples = train_data.num_examples

n_out = [b_neck]

In [43]:
# reset_tf_graph()

# if do_lwgan:
    
#     synthetic_data_out_dir = osp.join(top_data_dir, 'OUT/iclr/synthetic_samples/', experiment_tag)
#     create_dir(synthetic_data_out_dir)
    
#     lam = 5
#     beta = 0.5
#     reset_tf_graph()
#     gan = RawGAN_GP(experiment_tag, init_lr, lam, n_out, noise_dim, \
#                     latent_code_discriminator, latent_code_generator, \
#                     beta=beta, gen_kwargs={'layer_sizes': [64, 128], 'b_norm': False}, 
#                     disc_kwargs={'layer_sizes': [512, 1024], 'b_norm': False})

# else:
#     beta = 0.2
#     gan = LatentGAN(experiment_tag, init_lr, n_out, noise_dim, latent_code_discriminator_two_layers,
#                     latent_code_generator_two_layers, beta=beta, gen_kwargs={'b_norm': False}, 
#                     disc_kwargs={'b_norm': False})

In [40]:
beta = 0.5
lam_range = [12, 14, 16, 18]

for lam in lam_range:
    print lam
    synthetic_data_out_dir = osp.join(top_data_dir, 'OUT/iclr/synthetic_samples/', experiment_tag, 'lam_' + str(lam))
    create_dir(synthetic_data_out_dir)
    reset_tf_graph()
    gan = RawGAN_GP(experiment_tag, init_lr, lam, n_out, noise_dim, \
                    latent_code_discriminator, latent_code_generator, \
                    beta=beta, gen_kwargs={'layer_sizes': [64, 128], 'b_norm': False}, 
                    disc_kwargs={'layer_sizes': [512, 1024], 'b_norm': False})
    
    if do_training:
        for _ in range(max_epochs):
            loss, duration = gan._single_epoch_train(train_data, batch_size, noise_params)
            epoch = int(gan.sess.run(gan.epoch.assign_add(tf.constant(1.0))))
#             print epoch, loss

            if save_model and (epoch % saver_step == 0 or epoch <= 5):
                checkpoint_path = osp.join(train_dir, MODEL_SAVER_ID)
                gan.saver.save(gan.sess, checkpoint_path, global_step=gan.epoch)

            if epoch in save_synthetic_samples:
                syn_latent_data = gan.generate(n_syn_samples, noise_params)
                syn_data = ae.decode(syn_latent_data)
                np.savez(osp.join(synthetic_data_out_dir, 'epoch_' + str(epoch)), syn_data)
#                 for k in range(2):
#                     Point_Cloud(syn_data[k]).plot()

#             train_stats.append((epoch,) + loss)

12
14
16
18


In [34]:
if do_plotting:
    x = range(len(train_stats))
    d_loss = [t[1] for t in train_stats]
    g_loss = [t[2] for t in train_stats]
    plt.plot(x, d_loss, '--')
    plt.plot(x, g_loss)
    plt.title('Latent GAN training. (%s, %s)' %(class_name, ae_loss))
    plt.legend(['Discriminator', 'Generator'], loc=0)
    
    plt.tick_params(axis='x', which='both', bottom='off', top='off')
    plt.tick_params(axis='y', which='both', left='off', right='off')
    
    plt.xlabel('Epochs.') 
    plt.ylabel('Loss.')

In [38]:
voxel_resolution = 28
cmp_in_sphere = True
train_data_ = pclouds_centered_and_half_sphere(in_data['train'].point_clouds)
_, train_grid_var = entropy_of_occupancy_grid(train_data_, voxel_resolution, in_sphere=cmp_in_sphere)
test_data_ = pclouds_centered_and_half_sphere(in_data['test'].point_clouds)
_, test_grid_var = entropy_of_occupancy_grid(test_data_, voxel_resolution, in_sphere=cmp_in_sphere)

In [41]:
for lam in lam_range:
    print lam
    synthetic_data_out_dir = osp.join(top_data_dir, 'OUT/iclr/synthetic_samples/', experiment_tag, 'lam_' + str(lam))
    
    for epoch in save_synthetic_samples:
        in_f = osp.join(synthetic_data_out_dir, 'epoch_' + str(epoch) + '.npz')
        our_data = np.load(in_f)
        our_data = our_data[our_data.keys()[0]]
        our_data = pclouds_centered_and_half_sphere(our_data)    
        _, our_grid_var = entropy_of_occupancy_grid(our_data, voxel_resolution, in_sphere=cmp_in_sphere)
        jsd_epoch_train = jensen_shannon_divergence(our_grid_var, train_grid_var)
        jsd_epoch_test = jensen_shannon_divergence(our_grid_var, test_grid_var)    
        print 'NB-GAN', epoch, jsd_epoch_train, jsd_epoch_test

12
NB-GAN 1 0.13294844311 0.1317307377
NB-GAN 5 0.108942214483 0.108522532868
NB-GAN 10 0.0748141189945 0.0762263513816
NB-GAN 100 0.0133718704793 0.0163459607066
NB-GAN 200 0.0107807016904 0.0142246924581
NB-GAN 300 0.00807118503727 0.0120968935511
NB-GAN 400 0.00769829028966 0.0115881442461
NB-GAN 500 0.0074620278497 0.0111868022626
NB-GAN 600 0.00579280407089 0.0102500333158
NB-GAN 700 0.00656208540935 0.0105997457625
NB-GAN 800 0.00685257320906 0.0107173614015
NB-GAN 900 0.00586731716399 0.00977160209375
NB-GAN 1000 0.00572275026351 0.00967685611126
NB-GAN 1100 0.00605097122248 0.00952648436176
NB-GAN 1200 0.00626838847251 0.00991693202356
NB-GAN 1300 0.00578518583373 0.00991424122403
NB-GAN 1400 0.00583857190168 0.00936841504813
NB-GAN 1500 0.00618681490407 0.0100646229363
NB-GAN 1600 0.00546620620937 0.00942216913886
NB-GAN 1700 0.00554477154071 0.00912126164651
NB-GAN 1800 0.00592461656829 0.00950283204591
NB-GAN 1900 0.0063379943466 0.009460697158
NB-GAN 2000 0.00528060198034 0